In [1]:
from bs4 import BeautifulSoup
import re

import spacy
spacy.load("en")
import en_core_web_sm
nlp = en_core_web_sm.load()

In [2]:
html = open('/home/prashant/Desktop/QTR4/2488_000119312509245120_dex101.htm')
bs = BeautifulSoup(html, "html.parser")

In [3]:
titles = set()
sentences = []

In [4]:

for i,li in enumerate(bs.select('h1')):
        titles.add(li.text)
for i,li in enumerate(bs.select('h2')):
        titles.add(li.text)
        

para = []
for i, li in enumerate(bs.select('p')):
#     print("here2")
    bold = li.select('b')
    underline = li.select('u')
    italic = li.select('i')
    if bold != []:
        for j in range(len(bold)):
#             print(underline[j])
            if len(bold[j].text.split()) > 1:
                titles.add(str(bold[j].text).replace("\n", " "))
    if underline != []:
#         print(underline)
        for j in range(len(underline)):
#             print(underline[j])
            if len(underline[j].text.split()) > 1:
#                 print(underline[j].text)
                titles.add(str(underline[j].text).replace("\n", " "))
    if italic !=[]:
        for j in range(len(italic)):
#             print(underline[j])
            if len(italic[j].text.split()) > 1:
                titles.add(str(italic[j].text).replace("\n", " "))       
    else:
        para.append(li.text)

if para==[]:
    for i,li in enumerate(bs.select('font')):
        if 'FONT-WEIGHT: bold;' in str(li) or 'TEXT-DECORATION: underline;' in str(li) or 'FONT-STYLE: italic;' in str(li):
            if len(li.text.split()) <=25:
                li.text.replace(u'\xa0', u' ')
                titles.add(li.text.replace('\n',' '))
        else:
            li.text.replace(u'\xa0', u' ')
            para.append(li.text)

In [5]:
titles

{'Amendments and Waivers. ',
 'Counterpart to Registration Rights Agreement ',
 'Entire Agreement; Severability.',
 'Exhibit 10.1 ',
 'Governing Law. ',
 'Indemnification and Contribution',
 'No Inconsistent Agreements.',
 'Participation of Broker-Dealers in Exchange Offer',
 'Registration Procedures',
 'Registration Under the Securities Act',
 'Successors and Assigns. ',
 'Third Party Beneficiaries.',
 'pro rata',
 'provided, however',
 'provided, however,'}

In [6]:

for par in para:
    lines = nlp(par)
    for line in lines.sents:
        flag = 1
        for i in str(line).split():
            if not i.isupper():
                flag = 0
                break
        if flag and len(str(line).split()) > 1:
            titles.add(str(line).replace("\n"," "))
        else:
            tt = str(line).replace("\n"," ")
            if len(tt.split()) > 1:
                sentences.append(tt)
    

In [7]:
def get_pos_tags(sent):
    c = []
    c.append(str(sent).replace("\n"," "))
    c+=[0,0,0,0] # noun, adj, verb
    for word in sent:
        d = word.pos_
        if d == "VERB":
            c[3] += 1
        elif d in ["NOUN", "PROPN"]:
            c[1]+=1
        elif d == "ADJ":
            c[2]+=1
        else:
            c[4]+=1
    return c

In [8]:
data = []

In [9]:
m = []
for sent in titles:
    b = get_pos_tags(nlp(sent))
    x1 = b[1]
    x2 = b[2]
    x3 = b[3]
    x4 = b[4]
    total = x1+x2+x3+x4
    total1 = x1+x2+x3+x4*0.5
    if (x3)/total <= 0.20 and (x1+x2)/total1>=0.6:
#         print(b,(x1+x2)/total1)
        m.append(b[0])
        b.append(1)
        data.append(b)
    else:
        b.append(0)
        sentences.append(b[0])
        data.append(b)

titles = set(m)


In [10]:
for sent in sentences:
    b = get_pos_tags(nlp(sent))
    b.append(0)
    data.append(b)
    

In [11]:
data

[['No Inconsistent Agreements.', 2, 0, 0, 2, 1],
 ['Registration Under the Securities Act', 3, 0, 0, 2, 1],
 ['pro rata', 1, 1, 0, 0, 1],
 ['Entire Agreement; Severability.', 3, 0, 0, 2, 1],
 ['Participation of Broker-Dealers in Exchange Offer', 5, 0, 0, 3, 1],
 ['Exhibit 10.1 ', 1, 0, 0, 1, 1],
 ['Third Party Beneficiaries.', 3, 0, 0, 1, 1],
 ['ADVANCED MICRO DEVICES, INC.     ', 4, 0, 0, 3, 1],
 ['Indemnification and Contribution', 2, 0, 0, 1, 1],
 ['Successors and Assigns. ', 2, 0, 0, 2, 1],
 ['REGISTRATION RIGHTS AGREEMENT', 3, 0, 0, 0, 1],
 ['Registration Procedures', 2, 0, 0, 0, 1],
 ['Amendments and Waivers. ', 2, 0, 0, 2, 1],
 ['Governing Law. ', 1, 0, 1, 1, 0],
 ['provided, however,', 0, 0, 1, 3, 0],
 ['Counterpart to Registration Rights Agreement ', 4, 0, 0, 1, 1],
 ['provided, however', 0, 0, 1, 2, 0],
 ['Exhibit 10.1', 1, 0, 0, 1, 0],
 ['This REGISTRATION RIGHTS AGREEMENT dated November\xa030, 2009 (the “Agreement”) is entered into by and among Advanced Micro Devices, Inc.,

In [12]:
import random 
random.shuffle(data)

In [13]:
from pandas import DataFrame

In [14]:
df = DataFrame(data, columns= ["sent", "noun", "adj", "verb", "others", "label"])

In [15]:
df.head()

,sent,noun,adj,verb,others,label
0,(xii) cause the Indenture to be qualified unde...,31,7,24,59,0
1,"As used in this Agreement, the following terms...",3,0,5,7,0
2,"(vii) in the case of a Shelf Registration, upo...",17,6,4,34,0
3,(xiv) if reasonably requested by any Holder of...,21,12,13,34,0
4,In the event that the Company receives a Shelf...,47,11,10,78,0


In [16]:
len(sentences),len(titles)

(190, 14)

In [17]:
df.to_csv ('export_dataframe.csv', columns = ["sent", "noun", "adj", "verb", "others", "label"], index=True)

In [18]:
df.columns

Index(['sent', 'noun', 'adj', 'verb', 'others', 'label'], dtype='object')

In [19]:
df.head()

,sent,noun,adj,verb,others,label
0,(xii) cause the Indenture to be qualified unde...,31,7,24,59,0
1,"As used in this Agreement, the following terms...",3,0,5,7,0
2,"(vii) in the case of a Shelf Registration, upo...",17,6,4,34,0
3,(xiv) if reasonably requested by any Holder of...,21,12,13,34,0
4,In the event that the Company receives a Shelf...,47,11,10,78,0


In [20]:
q = set()
for i in titles:
    q.add(i.strip())

In [21]:
q

{'ADVANCED MICRO DEVICES, INC.',
 'Amendments and Waivers.',
 'Counterpart to Registration Rights Agreement',
 'Entire Agreement; Severability.',
 'Exhibit 10.1',
 'Indemnification and Contribution',
 'No Inconsistent Agreements.',
 'Participation of Broker-Dealers in Exchange Offer',
 'REGISTRATION RIGHTS AGREEMENT',
 'Registration Procedures',
 'Registration Under the Securities Act',
 'Successors and Assigns.',
 'Third Party Beneficiaries.',
 'pro rata'}